In [2]:
pip install gensim


  Using cached gensim-4.3.2-cp311-cp311-win_amd64.whl.metadata (8.5 kB)
Using cached gensim-4.3.2-cp311-cp311-win_amd64.whl (24.0 MB)


In [13]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
df=pd.read_csv("extracted_entities_resume.csv")

In [15]:
print(df.columns.to_list())

['Highest degree', 'Degrees', 'Major', 'Skill']


In [16]:
skill_resume_values = df['Skill'].tolist()
print(skill_resume_values)

['jenkins, prometheus, grafana, sonarqube, junit, openstack, deployment, web development, spring boot, angular, containers, kubernetes, docker, ansible, git, mysql, java, symfony, html, css, agile project management, groovy, design, azure, terraform, windows, debian, ubuntu, github, gitlab, .net, operating system, code, programming languages, database, engineering, computer architecture, communications']


In [18]:
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [23]:
# Example lists of skills
skills_list1 = ['python', 'machine', 'learning', 'data', 'analysis']
skills_list2 = ['python', 'machine', 'learning', 'data', 'analysis']

In [28]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
# Load the SentenceTransformer model
model = SentenceTransformer('all-mpnet-base-v2')

# Save the model
model.save('sentence_transformer_model')

In [30]:
def semantic_similarity_sbert_base_v2(skills_list1,skills_list2):
    """calculate similarity with SBERT all-mpnet-base-v2"""
    model_path = "sentence_transformer_model"
    model = SentenceTransformer(model_path)
    #Encoding:
    score = 0
    sen = skills_list1+skills_list2
    sen_embeddings = model.encode(sen)
    for i in range(len(skills_list1)):
        if skills_list1[i] in skills_list2:
            score += 1
        else:
            if max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(skills_list1):])[0]) >= 0.4:
                score += max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(skills_list1):])[0])
    score = score/len(skills_list1)  
    return round(score,3)

In [31]:
print(semantic_similarity_sbert_base_v2(skills_list1,skills_list2))

0.4


In [22]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



# Load Word2Vec model (pre-trained or trained on your data)
# Make sure to provide the correct path to your Word2Vec model file
word2vec_model = Word2Vec.load("word2vec.model")

# Tokenize skills and generate embeddings
def get_embeddings(skill_list):
    embeddings = []
    for skill in skill_list:
        if skill in word2vec_model.wv:
            embeddings.append(word2vec_model.wv[skill])
    return np.array(embeddings)

# Filter out skills without embeddings
embeddings1 = get_embeddings(skills_list1)
embeddings2 = get_embeddings(skills_list2)

# Check if embeddings lists are not empty
if embeddings1.size == 0 or embeddings2.size == 0:
    print("One of the skills lists does not contain any embeddings.")
else:
    # Aggregate token embeddings
    agg_embedding1 = np.mean(embeddings1, axis=0)
    agg_embedding2 = np.mean(embeddings2, axis=0)

    # Calculate cosine similarity
    similarity = cosine_similarity([agg_embedding1], [agg_embedding2])[0][0]
    print("Similarity between the two lists of skills:", similarity)



One of the skills lists does not contain any embeddings.
